## Synthetic Data Generation Experiment

Verwendet man das Qwen2.5-Coder-14B-Instruct Modell statt Qwen2.5-Coder-7B-Instruct zeigen sich erhebliche Verbesserungen, bei der Generierung und Reparatur von Code. Anstatt Codebeispiele für ein Fine-Tuning des kleineren Modells mittels der unter nicht-kommerziellen Creative Commons Lizenz stehenden Videocodes zu verwenden, kann ein bestehendes größeres Qwen2.5-Coder Modell (etwa 14B oder 32B) verwendet werden, um in der Pipeline den Code für einige Beispiele zu generieren, die dann genutzt werden können, um das kleine Modell zu verbessern und potenziell auf das gleiche Niveau zu heben.

Dafür müssen zunächst synthetisch Textaufgaben generiert werden, die dann im Video Planning Teil der Pipeline in einen Video Plan und daraufhin in dem Code Generation Teil der Pipeline zur Generierung von validem Code verwendet werden kann, der qualitativ höherwertiger ist als der des originalen 7B Modells.

Da die Ausführung der Pipeline mit großen Modellen sehr viel Zeit und Ressourcen benötigt, wurde hier lediglich der erste Schritt der Aufgabengenerierung skizziert, welche mit der bestehenden Pipeline verwendet werden können.

Um ein gutes Modell im Fine-Tuning zu erhalten, müssten die Prompts besser optimiert sein und auch Trainingsdaten für die anderen Schritte generiert werden, da in der aktuellen Pipeline das Code LLM mehrere Einsatzzwecke hat, in denen es nicht durch Fine-Tuning in eine Richtung Qualität an anderer Stelle einbüssen soll.

In [1]:
# Ollama LLM
import ollama

# Utility
from pydantic import BaseModel
from typing import List
import json

### Generate Tasks

Der wichtigste Teil ist es Textaufgaben, die letztlich durch die Videopipeline visualisiert werden können, zu generieren. Hierbei kann Structured Output Generation helfen, um eine Liste mit verschiedenen Aufgaben zu generieren.

In [2]:
# Define Tasks Model
class Tasks(BaseModel):
    tasks: List[str]

generate_tasks_system_prompt = """
You are an expert math educator focused in creating very well learning tasks.

The tasks should be text tasks where concrete numbers are given.

The tasks should be written in a real-world scenario.

Only generate the tasks and nothing else.

The format should be "Task: [...]" where [...] is a placeholder for the generated task.
"""

generate_tasks_user_prompt = """
Come up with ten different learning tasks on geometric problems.
"""

response = ollama.chat(
    model="phi4",
    messages=[
        {
            "role": "system",
            "content": generate_tasks_system_prompt
        },
        {
            "role": "user",
            "content": generate_tasks_user_prompt
        }
    ],
    format=Tasks.model_json_schema()
)

In [3]:
# Print generated tasks
tasks = json.loads(response.message.content)
tasks["tasks"]

["Task: Sarah's garden plot is in the shape of a rectangle. The length of her garden is 15 meters, and the width is 10 meters. Calculate the area of Sarah's garden.\n\n",
 'Task: A circular pizza has a diameter of 16 inches. What is the circumference of the pizza? Use π = 3.14 for your calculations.\n\n',
 'Task: John wants to build a square patio in his backyard. Each side of the square will be 8 feet long. How much material does he need to cover the entire patio?\n\n',
 'Task: A triangular plot of land has sides measuring 9 meters, 12 meters, and 15 meters. Is this triangle a right triangle? Explain your reasoning.\n\n',
 'Task: A cylindrical water tank has a radius of 3 meters and a height of 5 meters. What is the volume of the water tank?\n\n',
 'Task: Emma is designing a rectangular picture frame with an outer length of 18 inches and width of 12 inches. The frame itself should be 2 inches wide all around. Calculate the area of the glass needed for the picture.\n\n',
 'Task: A regu

### Generate Task Variations

Um die Anzahl der Trainingsbeispiele zu erhöhen, kann das gleiche Verfahren wie oben verwendet werden, allerdings mit der Änderung, dass für einen der bereits generierten Tasks Variationen generiert werden sollen.

In [4]:
class TaskVariations(BaseModel):
    variations: List[str]

generate_variations_system_prompt = """
You are an expert math educator focused in creating very well variations of learning tasks.

Adapt the scenario and the numbers.

The format should be "Variation: [...]" where [...] is a placeholder for the generated variation of the task.
"""

generate_variations_user_prompt = """
Come up with ten different variations of this task on a geometric problem: {task}
"""

task_dict = {

}

for task in tasks["tasks"]:
    response = ollama.chat(
        model="phi4",
        messages=[
            {
                "role": "system",
                "content": generate_variations_system_prompt
            },
            {
                "role": "user",
                "content": generate_variations_user_prompt.format(task=task)
            }
        ],
        format=TaskVariations.model_json_schema()
    )

    variations = json.loads(response.message.content)
    task_dict[task] = variations["variations"]

In [5]:
# Print number of tasks
print(len(task_dict.keys()))

10


In [6]:
# Print tasks and variations
for key, items in task_dict.items():
    print(key)
    print()
    print("Number variations: ", len(items))
    for item in items:
        print(item)
    print()

Task: Sarah's garden plot is in the shape of a rectangle. The length of her garden is 15 meters, and the width is 10 meters. Calculate the area of Sarah's garden.



Number variations:  10
Variation: John's backyard swimming pool is rectangular. It measures 20 meters in length and 12 meters in width. Determine the area of John's swimming pool.
Variation: Emma has a rectangular piece of land for her vegetable garden that is 18 meters long and 9 meters wide. Find out how much space she has to plant vegetables.
Variation: A rectangular parking lot has dimensions of 25 meters by 14 meters. Compute the area of this parking lot.
Variation: The floor of a classroom is rectangular, with a length of 16 meters and a width of 11 meters. What is the total area covered by the classroom floor?
Variation: A farmer's field is in the shape of a rectangle. It measures 30 meters along its longer side and 20 meters across its shorter side. Calculate how much land the farmer owns.
Variation: The rectangula

In einem nächsten Schritt würden dann die Videopläne sowie der Code generiert werden, die dann in dem Prompt Template zu einem Trainingsdatendatensatz zusammengeführt werden können, um anschließend zum Fine-Tuning des Modells genutzt werden zu können.

Das Fine-Tuning des Qwen2.5-Coder-7B-Instruct Modells wurde in den Notebooks `Model_Fine_Tuning.ipynb` mit dem Alpaca Datensatz und `unsloth.ipynb` skizziert. Der Code basiert größtenteils auf dem Unsloth Beispiel Colab für Qwen2.5 Modelle.